In [17]:
%load_ext cython

In [18]:
import matplotlib.pyplot as plt
import math
import numpy as np

In [23]:
%%cython -a -+ 
cimport cython
import numpy as np
cimport numpy as np
from scipy import optimize

from libc.math cimport exp, tanh
from mc_lib.rndm cimport RndmWrapper
from mc_lib.lattices import tabulate_neighbors
from mc_lib.observable cimport RealObservable

    
cdef void init_spins(long[::1] spins,  RndmWrapper rndm): 
    
    for j in range(spins.shape[0]):
        spins[j] = 1 if rndm.uniform() > 0.5 else -1
        
        
        
@cython.boundscheck(False)
@cython.wraparound(False)
cdef double energy(long[::1] spins, 
                   long[:, ::1] neighbors,
                  const double[:,::1] Js):

    cdef:
        double ene = 0.0
        Py_ssize_t site, site1, num_neighb

    for site in range(spins.shape[0]):
        num_neighb = neighbors[site, 0]
        for j in range(1, num_neighb+1):
            site1 = neighbors[site, j]
            ene += -1 * Js[site, site1] * spins[site] * spins[site1] 
    
    return ene / 2.0



@cython.boundscheck(False)
@cython.wraparound(False)
cdef void flip_spin(long[::1] spins, 
                    const long[:, ::1] neighbors,
                    double beta,
                    const double[:,::1] Js, RndmWrapper rndm): 
    cdef:
        Py_ssize_t site = int(spins.shape[0] * rndm.uniform())
        Py_ssize_t site1

    cdef long num_neighb = neighbors[site, 0]
    cdef double summ = 0.
    for j in range(1, num_neighb + 1):
        site1 = neighbors[site, j]
        summ += spins[site1] * spins[site] * Js[site,site1]
   
    cdef double ratio = exp(-2.0 * beta * summ )
    
    if rndm.uniform() > ratio:
        return

    spins[site] = -spins[site]
    

    
def get_a_time(int L, double T, double Jd, T_c):
    
    Ls = (10,20,30,40,60,80)
    tau = dict(zip(Ls,(20,40, 60, 90, 200, 350)))
    for i in Ls:
        if L <= i:
            l = L
            
    if T <= (T_c - 0.05):
        a_time = 25
    if T < (T_c + 0.05) and T > (T_c - 0.05):
        a_time = tau[l]
    if T >= (T_c + 0.05) and T<(T_c + 0.1):
        a_time = 50
        if l == 10:
            a_time = 20
    if T >=(T_c+0.1) and T <(T_c+0.2):
        a_time = 15
    if T >= (T_c + 0.2):
        a_time = 1
    return a_time

def get_T_c(double Jd):
    F = lambda t: (np.sinh(2/t))**2 + 2 * np.sinh(2/t) * np.sinh(Jd *2/t) - 1
    sol=optimize.root(F,1).x[0]
    return sol
    
cdef void get_J( double[:,::1] Js, double J, double Jd, int L1, int L2 , int L3 = 1):
  
    if L3 == 1:
        for i in range(L1*L2):
            Js[i, ((i // L2 + 1) % L1 * L2 )  + (i + 1) % L2 ] = Jd
            Js[i, ((i // L2  - 1) % L1 * L2 )  + (i - 1) % L2 ] = Jd
            Js[i, (i // L2) * L2 + (i + 1) % L2] = J 
            Js[i, (i + L2) % (L1*L2)] = J
            Js[i, (i // L2) * L2 + (i - 1) % L2] = J
            Js[i, (i - L2) % (L1*L2)] = J
        return
    
    else:
        return


def simulate(Py_ssize_t L,
             double T, double J, double Jd,
             Py_ssize_t num_sweeps, int seed):

    cdef:
        long[:, ::1] neighbors = tabulate_neighbors(L, kind='triang') 
        double beta = 1./T

    cdef:
        
        int num_therm = int(30 * L)
        int steps_per_sweep = 2 * L * L
        int sweep = 0
        int i
        
    cdef RndmWrapper rndm = RndmWrapper((1234, seed)) 
    
    cdef long[::1] spins =  np.empty( L*L, dtype=int) 
    init_spins(spins, rndm)
    
    cdef double[:,::1] Js = np.zeros((L*L, L*L)) 
    get_J(Js, J, Jd, L, L)
    
    spins_set = np.empty((num_sweeps, L*L), dtype = int)
    
    T_c = get_T_c(Jd)
    a_time = get_a_time(L,T,Jd,T_c)

    for sweep in range(num_therm):
        for i in range(steps_per_sweep):
            flip_spin(spins, neighbors, beta, Js, rndm)

    for sweep in range(num_sweeps):
        for i in range(a_time * steps_per_sweep):
            flip_spin(spins, neighbors, beta, Js, rndm)
        spins_set[sweep] = spins
    
 
    return spins_set
  

In [4]:
from tqdm import tqdm
import random

In [6]:
sizes = [20, 30, 40, 50 , 60, 80, 100] #размеры решеток
temps = np.arange(0.05, 4, 0.2) # температуры
vol = 2048 # объемы выборок
Jd_list = [-0.1, -0.3] # фрустрации

for Jd in Jd_list:
    for size in tqdm(sizes):
        dataset = np.zeros((len(temps), vol, size**2 ), dtype = np.int8) # vol конфигураий для каждой температуры
        for i,t in enumerate(temps):
            seed = random.randint(0,2000)
            dataset[i] = simulate(size, t, 1, Jd, vol, seed)
    
        np.save(f'conf_{size}_{Jd}.npy', dataset) # данные сохраняются в такие файлы


100%|██████████| 7/7 [21:27<00:00, 183.90s/it]

100%|██████████| 7/7 [17:47<00:00, 152.48s/it]
